In [4]:
import pandas as pd, numpy as np
from get_draws.api import get_draws
from db_queries import get_population, get_ids, get_location_metadata, get_outputs

In [2]:
l = get_location_metadata(location_set_id=1, gbd_round_id=6)
subnationals = l.loc[l.parent_id==179]
subnational_ids = list(subnationals.location_id.unique())
subnational_ids

[44861, 44853, 44854, 44857, 44862, 44860, 44859, 44855, 44856, 44858, 44852]

In [3]:
[179] + subnational_ids

[179,
 44861,
 44853,
 44854,
 44857,
 44862,
 44860,
 44859,
 44855,
 44856,
 44858,
 44852]

In [ ]:
df = get_draws(gbd_id_type='rei_id',
                 gbd_id=240,
                 source='exposure',
                # year_id=[2012,2019],
                 gbd_round_id=6,
                 status='best',
                 location_id = [179] + subnational_ids,
                 decomp_step = 'step4')
df = (df.loc[(df.parameter.isin(['cat1','cat2']))
           &(df.age_group_id.isin([4,5]))]
      .groupby(['location_id','year_id','sex_id','age_group_id']).sum())
df = df.drop(columns=[c for c in df.columns if 'draw' not in c])
df

In [ ]:
pop = get_population(gbd_round_id=6,
                    year_id=list(df.reset_index().year_id.unique()),
                    location_id=[179] + subnational_ids,
                    sex_id=[1,2],
                    age_group_id=[4,5],
                    decomp_step='step4')[['location_id','year_id','age_group_id','sex_id','population']]
pop

In [ ]:
data = df.reset_index().merge(pop, on=['age_group_id','sex_id','location_id','year_id'])
for i in list(range(0,1000)):
    data[f'draw_{i}'] = data[f'draw_{i}'] * data.population
data = data.groupby(['location_id','year_id']).sum()
for i in list(range(0,1000)):
    data[f'draw_{i}'] = data[f'draw_{i}'] / data.population
data = (data.drop(columns=[c for c in data.columns if 'draw' not in c])
        .apply(pd.DataFrame.describe, axis=1, percentiles=[0.025,0.975]))
data

In [ ]:
d = data.reset_index()
d.loc[d.year_id==2010].sort_values(by='mean').merge(subnationals[['location_id','location_name']], on='location_id')

In [5]:
bur = get_outputs(gbd_round_id=6,
                  year_id=2019,
                  age_group_id=1,
                  location_id=179,
                  topic='rei',
                  rei_id=240,
                  decomp_step='step5')
bur

,age_group_id,cause_id,location_id,measure_id,metric_id,rei_id,sex_id,year_id,acause,age_group_name,...,location_name,location_type,measure_name,metric_name,rei,rei_name,sex,val,upper,lower
0,1,294,179,1,1,240,3,2019,_all,Under 5,...,Ethiopia,admin0,Deaths,Number,nutrition_wasting,Child wasting,Both,35488.721681,49945.866043,24738.812395


In [6]:
dalys = get_outputs(gbd_round_id=6,
                  year_id=2019,
                  age_group_id=1,
                  location_id=179,
                  topic='cause',
                  cause_id=294,
                  decomp_step='step5')
dalys

,age_group_id,cause_id,location_id,measure_id,metric_id,sex_id,year_id,acause,age_group_name,cause_name,expected,location_name,location_type,measure_name,metric_name,sex,val,upper,lower
0,1,294,179,1,1,3,2019,_all,Under 5,All causes,False,Ethiopia,admin0,Deaths,Number,Both,190172.915684,242574.96096,149788.722632


In [7]:
35488.721681 / 190172.915684

0.1866129125346623